In [1]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


from scipy.ndimage import maximum_filter as maxf2D
from scipy.ndimage import minimum_filter as minf2D
from scipy.ndimage import convolve as conf2D
# https://stackoverflow.com/questions/43966350/getting-the-maximum-in-each-rolling-window-of-a-2d-numpy-array

import numpy as np
#import matplotlib.pyplot as plt
#import rioxarray as rioxr
#import geopandas as gpd
#import pandas as pd
import rasterio
#import shapely
#import os

import pystac_client
import planetary_computer as pc

In [3]:
# ------------------------------------------------------------------------------
# fp = filepath of csv, must have x and y columns representing coordinates of point
# crs must be the crs of the csv coordinates
def geodataframe_from_csv(fp, crs):
    df_raw = pd.read_csv(fp)
    df = df_raw.drop(['geometry','Unnamed: 0'], axis=1)
    
    # recreate geometry column with shapely Points
    xy = []
    for x,y in zip(df.x,df.y):
        xy.append(Point(x,y))
    df['geometry'] = xy
    df = df.drop(['x','y'], axis=1)

    pts = gpd.GeoDataFrame(df, crs=crs)
    return pts

# ------------------------------------------------------------------------------



In [4]:
# accesing Azure storage using pystac client
URL = "https://planetarycomputer.microsoft.com/api/stac/v1"
catalog = pystac_client.Client.open(URL)

#campus point 2018 naip scene
itemid = 'ca_m_3411934_sw_11_060_20180722_20190209'
search = catalog.search(
    collections=["naip"],
    ids = itemid
)
item = list(search.get_items())[0]
# sign and open item
href = pc.sign(item.assets["image"].href)
ds = rasterio.open(href)
ds.crs

CRS.from_epsg(26911)

In [5]:
fp = '/home/jovyan/msai4earth-esa/iceplant_detection/data_sampling/points_from_naip_images/campus_lagoon_points/campus_lagoon_points_2018.csv'
pts = geodataframe_from_csv(fp, ds.crs)

/srv/conda/envs/notebook/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [7]:
pts.head(3)

,iceplant,r,g,b,nir,year,month,day,naip_id,polygon_id,geometry
0,1,93,88,93,141,2018,7,22,ca_m_3411934_sw_11_060_20180722_20190209,0,POINT (238343.430 3810848.805)
1,1,85,85,89,137,2018,7,22,ca_m_3411934_sw_11_060_20180722_20190209,0,POINT (238343.547 3810846.093)
2,1,89,85,91,133,2018,7,22,ca_m_3411934_sw_11_060_20180722_20190209,0,POINT (238344.864 3810845.535)


In [8]:
fp = os.path.join(os.getcwd(),'3mlidar_over_campuspoint_naip_scene.tif')

lidar = rasterio.open(fp)
lidar.crs

CRS.from_epsg(32610)

In [26]:
lidar_rast = lidar.read([1])

In [22]:
def pts_for_lidar_sampling(raw_pts, lidar_crs):
    pts_xy = pts.to_crs(lidar_crs).geometry.apply(lambda p :(p.x, p.y)).to_list()
    return pts_xy

# ------------------------------------------------------------------------------

def sample_raster(pts_xy, raster):
    sample = raster.sample(pts_xy)
    samples = []
    for x in sample:
        samples.append(x[0])
    return samples



In [23]:
pts_xy = pts_for_lidar_sampling(pts, lidar.crs)

In [27]:
lidar_samples = sample_lidar(pts_xy, lidar_rast)

In [ ]:
def save_raster(fp, shape, bands_n, crs, transform, dtype):
    with rasterio.open(
            fp,  # file path
            'w',           # w = write
            driver='GTiff', # format
            height = shape[0], 
            width = shape[1],
            count = bands_n,  # number of raster bands in the dataset
            dtype = dtype,
            crs = crs,
            transform = transform,
        ) as dst:
            dst.write(preds.astype(dtype), bands_n)
    return 
    
# ------------------------------------------------------------------------------

# folder_path = path to folder to save rasters
def min_max_diff_avg_rasters(lidar_pts, lidar_rast_reader, folder_path):    
    rast = lidar_rast_reader.read([1]) # read raster
    
    mins = minf2D(rast, size=(3,3))    # calculate min max and difference
    maxs = maxf2D(rast, size=(3,3))
    diffs = maxs - mins
    
    # save rasters
    m = [maxs,mins,diffs]
    m_labels = ['maxs', 'mins', 'diffs']
    for i in range(0,3):
        fp = os.path.join(folder_path, 'lidar_'+m_labels +'.tif')
        save_raster(fp, rast.shape, lidar_rast_reader.crs, lidar_rast_reader.transform, rasterio.uint8):
    
    # calculate averages
    w = np.ones(9).reshape(3,3)
    avgs = conf2D(a, 
             weights=w,
             mode='constant')
    avgs = avgs/9
    
    # save averages
    fp = os.path.join(folder_path, 'lidar_avgs.tif')
    save_raster(fp, rast.shape, lidar_rast_reader.crs, lidar_rast_reader.transform, rasterio.float32):
    
    return

